In [1]:
import pandas as pd
from pathlib import Path
import csv

In [2]:
 #Import csv file
filepath = Path("Data/personal transaction.csv")

In [3]:
pd= pd.read_csv(filepath, index_col='Date', infer_datetime_format=True, parse_dates=True)
pd.head()

,Month,Description,Amount,Category,Account Name
Date,,,,,
2018-01-01,JANUARY,Amazon,11.11,Shopping,Platinum Card
2018-01-02,JANUARY,Mortgage Payment,1247.44,Mortgage & Rent,Checking
2018-01-02,JANUARY,Thai Restaurant,24.22,Restaurants,Silver Card
2018-01-04,JANUARY,Netflix,11.76,Movies & DVDs,Platinum Card
2018-01-05,JANUARY,American Tavern,25.85,Restaurants,Silver Card
